In [1]:
from __future__ import print_function
from future_builtins import map, filter, zip

In [2]:
import tensorflow as tf
import keras
import coremltools
import numpy as np

for name, package in [('tensorflow',tf), ('keras',keras), ('coremltools',coremltools), ('numpy', np)]:
    try:
        print(name + ' v' + package.__version__)
    except AttributeError:
        print(name + ' v?')

Using TensorFlow backend.


tensorflow v1.4.0
keras v2.1.2
coremltools v?
numpy v1.13.3


In [3]:
# Load newline-delimited list of categories

# f = open('quickdraw-categories.txt','r')
f = open('50-categories.txt','r')
categories = [line.rstrip() for line in f]
f.close()

filename_from_category = lambda s: '../../quickdraw/'+s+'.npy'
filenames = list(map(filename_from_category, categories))

In [4]:
# Test that all files exist according to categories
# in quickdraw-categories.txt

import os.path

all_exist = True
for filename in filenames:
    if not os.path.isfile(filename):
        print('file `{}` does not exist'.format(filename))
        all_exist = False
if all_exist:
    print('All {} files found!'.format(len(categories)))

All 50 files found!


In [5]:
image_size = (28,28)
samples_per_category = 30000

In [6]:
def one_hot_vector(index):
    hot = np.zeros(len(categories),dtype=np.int8)
    hot[index] = 1
    return hot

def one_hot_array(index):
    is_column = False
    hot = np.zeros((1,len(categories)),dtype=np.int8)
    hot[0,index] = 1
    if is_column:
        return hot.T
    else:
        return hot

In [7]:
import random

def make_dataset():
    data = []
    target = []
    for category_index, (category, filename) in enumerate(zip(categories, filenames)):
        label_one_hot_encoded = one_hot_vector(category_index)
        category_images = np.load(filename)
        n_images = category_images.shape[0]
        for image in category_images[:samples_per_category]:
            image.shape = image_size     #Make Square
            data.append(image)
            target.append(label_one_hot_encoded)
    data = np.array(data)
    data = np.expand_dims(data, axis=3)
    target = np.array(target)
    print(data.shape)
    print(target.shape)
    return (data, target)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Reshape, Dense, BatchNormalization, Dropout, \
                         Conv2D, MaxPooling2D, Activation, Flatten, Lambda

def _make_convolution_layers(model):
    model.add(
        Reshape(
            # input shape is height, width, channels
            (input_shape[0], input_shape[1], 1), 
            input_shape=(input_shape[0], input_shape[1], 1)
        )
    )
    
#     model.add(
#         Lambda(lambda x: x/127.5-1)
#     )

    for filters in n_filters:
        # Add a single convolution layer
        model.add(
            Conv2D(
                filters=filters,
                kernel_size=(3,3),
                padding='same',
                activation=layer_activation
            )
        )
        # Add batch normalization to the convolution layer
        if batch_normalize:
            model.add(
                BatchNormalization(axis=1)
            )
        # Pool the layer
        model.add(
            # channels_last is required for coremltools
            MaxPooling2D(pool_size=(2,2), data_format='channels_last')
        )

    model.add(Flatten())

def _make_dense_layers(model):
    for size in dense_sizes:
        model.add(
            Dense(size, activation=layer_activation)
        )
        if batch_normalize:
            model.add(
                BatchNormalization(axis=1)
            )
    model.add(
        Dense(output_size, activation=final_activation)
    )

def one_hot_output(y_hat_raw):
    # predict() outputs floats. We want one-hot in most cases
    y_hat = np.zeros(y_hat_raw.shape, dtype=np.int)
    y_hat[np.arange(y_hat_raw.shape[0]), y_hat_raw.argmax(1)] = 1
    return y_hat

In [9]:
data, target = make_dataset()

(1500000, 28, 28, 1)
(1500000, 50)


In [10]:
import os.path
from keras.models import load_model
import json

def cache_fit(model_name, model, *args, **kwargs):
    archive_name = model_name+'_model.h5'
    history_name = model_name+'_history.json'
    archive_exists = os.path.isfile(archive_name)

    if not archive_exists:
        print('Model '+model_name+' not found in archive. Training new model.')
        hist = model.fit(*args, **kwargs)
        model.save(archive_name)
        with open(history_name, 'w') as f:
            json.dump(hist.history, f)
        return model
    else:
        print('Model found on disk. Reloading.')
        return load_model(archive_name)

In [11]:
input_shape=image_size
n_filters=[64,64,128,128]
dense_sizes=[128,128]
output_size=len(categories)

batch_normalize = True

layer_activation='relu'
final_activation='softmax'

loss='categorical_crossentropy'
optimizer='Nadam'
metrics=['categorical_accuracy']

#########

model = Sequential()

_make_convolution_layers(model)
_make_dense_layers(model)

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=metrics
)

model.summary()

model_name = 'cnn_50cat_30000img_4ep'

model = cache_fit(
    model_name, model, 
    x=data, y=target, 
    batch_size=32, epochs=4, 
    verbose=1, shuffle=True
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        112       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 64)        56        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
__________

In [12]:
coreml_model = coremltools.converters.keras.convert(
    model, 
    input_names='drawing', 
    image_input_names='drawing', 
    class_labels='50-categories.txt'
)

coreml_model.save(model_name+'.mlmodel')

0 : reshape_1_input, <keras.engine.topology.InputLayer object at 0x7f70a06d31d0>
1 : reshape_1, <keras.layers.core.Reshape object at 0x7f70266b2c90>
2 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x7f70a06d3210>
3 : conv2d_1__activation__, <keras.layers.core.Activation object at 0x7f70a06882d0>
4 : conv2d_1_permute_batch_normalization_1, <keras.layers.core.Permute object at 0x7f70a0688410>
5 : batch_normalization_1, <keras.layers.normalization.BatchNormalization object at 0x7f70a065bb10>
6 : batch_normalization_1_permute_max_pooling2d_1, <keras.layers.core.Permute object at 0x7f70a06883d0>
7 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0x7f70266d1050>
8 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x7f6f53b00490>
9 : conv2d_2__activation__, <keras.layers.core.Activation object at 0x7f702043fe90>
10 : conv2d_2_permute_batch_normalization_2, <keras.layers.core.Permute object at 0x7f70a0688590>
11 : batch_normalization_2, <keras.layers.normalizat